## Relatório Final de Projeto

Integrantes: Emyle Santos, João Victor e Júlia Gabriele

Este projeto apresenta uma análise detalhada dos dados do **RREO** (Relatório Resumido da Execução Orçamentária), focando na interpretação e estudo de dados financeiros e orçamentários reportados pela entidade no primeiro bimestre de 2021 a 2023. Os dados foram fornecidos pela SINCONF (Supervisão de Controle Interno e Financeiro) e têm como objetivo fornecer insights sobre aspectos financeiros no Brasil durante esse período.

Os objetivos deste projeto são:

- Analisar as principais fontes de receita da entidade: Identificar e estudar as fontes de receita mais relevantes da entidade nos anos de 2021, 2022 e 2023, visando **entender as variações e tendências** ao longo desse período.

- Avaliar o comportamento das despesas: Realizar uma análise detalhada das despesas totais da entidade em cada ano, destacando as **categorias de despesa** mais significativas e suas variações.

- Examinar os principais investimentos: Identificar os principais investimentos realizados pela entidade durante o primeiro bimestre dos anos de 2021, 2022 e 2023, buscando avaliar seu impacto no **contexto financeiro.**

## Exploração Inicial

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
caminho_dados = r'C:\Users\gabri\OneDrive\Documentos\2 periodo\intro. cd\projetosiconf\dados\dados_totais.csv'
dataset = pd.read_csv(caminho_dados,  encoding="utf-8")
dataset.head()

,Ano,Instituição,UF,População,Conta,Identificador da Conta,Valor
0,2021,Prefeitura Municipal de Manaus - AM,AM,2255903,RECEITAS CORRENTES (I),siconfi-cor_ReceitasCorrentesLiquidasExcetoTra...,7.819839e+08
1,2021,Prefeitura Municipal de Manaus - AM,AM,2255903,"Impostos, Taxas e Contribuições de Melhoria",siconfi-cor_ReceitaTributariaLiquidaExcetoTran...,2.688272e+08
2,2021,Prefeitura Municipal de Manaus - AM,AM,2255903,IPTU,siconfi-cor_IPTULiquidoExcetoTransferenciasEFU...,1.491228e+08
3,2021,Prefeitura Municipal de Manaus - AM,AM,2255903,ISS,siconfi-cor_ISSLiquidoExcetoTransferenciasEFUNDEB,8.225389e+07
4,2021,Prefeitura Municipal de Manaus - AM,AM,2255903,ITBI,siconfi-cor_ITBILiquidoExcetoTransferenciasEFU...,8.944698e+06


In [4]:
# Criando uma copia do dataset para não modificar o original
dataset_copia = dataset.copy()

In [5]:
# Descrição estatística dos dados
dataset_copia.describe()

,Ano,População,Valor
count,30209.000000,3.020900e+04,3.020900e+04
mean,2021.976067,1.891848e+06,3.755082e+08
std,0.817692,2.528807e+06,2.490792e+09
min,2021.000000,3.062960e+05,-6.492568e+07
25%,2021.000000,5.489520e+05,5.250071e+06
50%,2022.000000,1.031597e+06,2.708876e+07
75%,2023.000000,1.963726e+06,1.441533e+08
max,2023.000000,1.239637e+07,8.657029e+10


Analisando a descrição estatísca, podemos retirar algumas informações sobre os dados:

*1*- Na coluna 'Ano', temos tres valores 2022, 2023, 2024, que correspondem aos anos que os dados foram retirados       
    1.1- A mediana é 2022, o que significa que metade dos dados correspondem ao ano de 2022

*2*- Na coluna 'População', a média é na casa dos milhões, sugerindo que os dados estão em uma escala grande        
    2.1- O desvio padrão de 2.528.807 sugere uma alta variação, com valores que se espalham amplamente em torno da média.

*3*- Na coluna 'Valor', a média também é um valor bem alto (na casa de centenas de milhões), o que sugere a alta escala de valores tambem.      
    3.1- Analisando o valor min e max, notamos a existência de valores negativos nos dados, o que faz sentido, levando em consideração
que a coluna traz informações sobre os pagamentos de contas.

In [6]:
# Vamos visualizar melhor as linhas referentes aos valores negativos da coluna "valor"
df_valores_negativos = dataset_copia[dataset_copia['Valor'] < 0]

df_valores_negativos

,Ano,Instituição,UF,População,Conta,Identificador da Conta,Valor
492,2021,Prefeitura Municipal de Maceió - AL,AL,1031597,( - ) Transferências obrigatórias da União rel...,siconfi-cor_RREO3TransferenciasObrigatoriasDaU...,-632793.00
535,2021,Prefeitura Municipal de Maceió - AL,AL,1031597,Outras Receitas Patrimoniais,siconfi-cor_OutrasReceitasPatrimoniais,-39174.87
582,2021,Prefeitura Municipal de Maceió - AL,AL,1031597,( - ) Transferências obrigatórias da União rel...,siconfi-cor_RREO3TransferenciasObrigatoriasDaU...,-596107.00
961,2021,Prefeitura Municipal de Fortaleza - CE,CE,2703391,(-) Transferências obrigatórias da União relat...,siconfi-cor_RREO3TransferenciasObrigatoriasDaU...,-33604.00
1049,2021,Prefeitura Municipal de Fortaleza - CE,CE,2703391,(-) Transferências obrigatórias da União relat...,siconfi-cor_RREO3TransferenciasObrigatoriasDaU...,-300000.00
...,...,...,...,...,...,...,...
28430,2023,Prefeitura Municipal de Belém - PA,PA,1499641,Rendimentos de Aplicação Financeira,siconfi-cor_RendimentosDeAplicacaoFinanceira,-1629428.92
28455,2023,Prefeitura Municipal de Belém - PA,PA,1499641,Receita Patrimonial,siconfi-cor_RREO3ReceitaPatrimonial,-1927275.69
28456,2023,Prefeitura Municipal de Belém - PA,PA,1499641,Rendimentos de Aplicação Financeira,siconfi-cor_RendimentosDeAplicacaoFinanceira,-1994966.02
28481,2023,Prefeitura Municipal de Belém - PA,PA,1499641,Receita Patrimonial,siconfi-cor_RREO3ReceitaPatrimonial,-68776.42


    Esses valores negativos podem indicar ajustes contábeis, como devoluções de transferências obrigatórias, correções em receitas e despesas, e situações de déficit orçamentário, onde despesas superam receitas previstas. 

## Pré-processamento de dados

## Perguntas

### Pergunta 1

### Pergunta 2

### Pergunta 3

## Conclusão